In [1]:
import os 
import mlflow

In [2]:
os.chdir("../")

In [3]:
%pwd

'c:\\Users\\DELL\\projects\\end-to-end-ML-with-MLflow'

In [4]:
os.environ["MLFOLOW_TRACKING_URI"]= "https://dagshub.com/Ahmad97/my-first-repo.mlflow" 
os.environ["DAGS_HUB_TOKEN"] =  "4cefc6718b7a65548a8d20d5c9012fcff949fd23"

In [5]:
import os

# Retrieve the token
token = os.environ.get("DAGS_HUB_TOKEN")

# Check if token exists
if token:
    print("Token found!")
else:
    print("Token not found. Please set the environment variable.")

Token found!


In [6]:
import requests
token= os.environ.get("DAGS_HUB_TOKEN")
uri = os.environ.get("MLFOLOW_TRACKING_URI")
def check_url_reachable(url):
    try:
        response = requests.get(url)
        # Check if the status code is 200 (OK)
        if response.status_code == 200:
            print(f"The URL {url} is reachable.")
        else:
            print(f"The URL {url} returned status code: {response.status_code}")
    except requests.exceptions.RequestException as e:
        print(f"An error occurred while trying to reach {url}: {e}")

# Example usage with your tracking URL
tracking_url = f"{uri}?token={token}" # Replace with your MLflow tracking URL
check_url_reachable(tracking_url)

The URL https://dagshub.com/Ahmad97/my-first-repo.mlflow?token=4cefc6718b7a65548a8d20d5c9012fcff949fd23 is reachable.


In [7]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelEvaluationConfig: 
    root_dir: Path
    test_data_path: Path
    model_path: Path
    metric_file_name: Path
    all_params: dict
    target_column: str 
    #mlflow_uri: str 




In [8]:
from mlproject.constants import * 
from mlproject.utils.common import read_yaml, create_directories, save_json

class ConfigurationManager:
    def __init__ (self,config_path= CONFIG_FILE_PATH, param_path =PARAMS_FILE_PATH, schema_path= SCHEMA_FILE_PATH):
        self.config = read_yaml(config_path)
        self.params = read_yaml(param_path)
        self.schema = read_yaml(schema_path)

        create_directories([self.config.artifacts_root])

    def get_configutaion_model_evaluation (self): 
        config = self.config.model_evaluation 
        params = self.params.ElasticNet
        schema = self.schema.TARGET_COLUMN 

        create_directories([config.root_dir])


        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path =config.test_data_path,
            model_path=config.model_path,
            all_params= params,
            metric_file_name= config.metric_file_name,
            target_column= schema.name,
            #mlflow_uri=f"{uri}?token={token}"
        )
        
        return model_evaluation_config


In [9]:
%pip install protobuf==3.20.1
%pip install dags

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [10]:
import os 
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn 
import numpy
import joblib 
import numpy as np


In [11]:
%pip install dagshub

Note: you may need to restart the kernel to use updated packages.


In [12]:
import dagshub


In [13]:
class ModelEvaluation: 
    def __init__(self, config: ModelEvaluationConfig): 
        self.config = config
        
    def eval_metrics(self, actual, pred): 
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae= mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)
        return rmse, mae, r2 
    
    def log_into_mlflow(self): 
        #print(self.config)
        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)
        
        x_test = test_data.drop([self.config.target_column], axis=1) 
        y_test = test_data[self.config.target_column]

       #mlflow.set_tracking_uri(self.config.mlflow_uri)
        #tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        #print(tracking_url_type_store)
        dagshub.init(repo_owner='Ahmad97', repo_name='my-first-repo', mlflow=True)
        with mlflow.start_run():
            y_predicted = model.predict(x_test)
            (rmse, mae, r2) = self.eval_metrics(y_test, y_predicted)
            scores = {"rmse": rmse, "mae": mae, "r2": r2}
            save_json(path=Path(self.config.metric_file_name), data=scores)
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics({"rmse":rmse, "mae":mae, "r2":r2})
            mlflow.sklearn.log_model(model, "model", registered_model_name="ElasticnetModel")

            #if tracking_url_type_store !="file":
                #mlflow.sklearn.log_model(model, "model", registered_model_name="ElasticnetModel")
            #else: 
                #mlflow.sklearn.log_model(model, "model")



In [14]:
try:
    config = ConfigurationManager()
    mode_evaluation_config= config.get_configutaion_model_evaluation()
    mode_evaluation = ModelEvaluation(config=mode_evaluation_config)
    mode_evaluation.log_into_mlflow()
except Exception as e : 
    raise e

[2024-12-31 06:08:56,104 - common - INFO - yaml file: config\config.yaml loadded successfully]
[2024-12-31 06:08:56,106 - common - INFO - yaml file: params.yaml loadded successfully]
[2024-12-31 06:08:56,108 - common - INFO - yaml file: schema.yaml loadded successfully]
[2024-12-31 06:08:56,109 - common - INFO - created directory at: artifacts]
[2024-12-31 06:08:56,110 - common - INFO - created directory at: artifacts/model_evaluation]
[2024-12-31 06:08:56,456 - _client - INFO - HTTP Request: GET https://dagshub.com/api/v1/user "HTTP/1.1 200 OK"]


Accessing as Ahmad97

[2024-12-31 06:08:56,462 - helpers - INFO - Accessing as Ahmad97]
[2024-12-31 06:08:56,782 - _client - INFO - HTTP Request: GET https://dagshub.com/api/v1/repos/Ahmad97/my-first-repo "HTTP/1.1 200 OK"]
[2024-12-31 06:08:57,095 - _client - INFO - HTTP Request: GET https://dagshub.com/api/v1/user "HTTP/1.1 200 OK"]


Initialized MLflow to track repo "Ahmad97/my-first-repo"

[2024-12-31 06:08:57,098 - helpers - INFO - Initialized MLflow to track repo "Ahmad97/my-first-repo"]


Repository Ahmad97/my-first-repo initialized!

[2024-12-31 06:08:57,100 - helpers - INFO - Repository Ahmad97/my-first-repo initialized!]
[2024-12-31 06:08:57,967 - common - INFO - json file saved at: artifacts\model_evaluation\metrics.json]


c:\Users\DELL\anaconda3\lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
Registered model 'ElasticnetModel' already exists. Creating a new version of this model...
2024/12/31 06:09:05 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: ElasticnetModel, version 2
Created version '2' of model 'ElasticnetModel'.


In [15]:
print("MLflow Tracking URI:", mlflow.get_tracking_uri())
os.environ.get("MLFOLOW_TRACKING_URI")

MLflow Tracking URI: https://dagshub.com/Ahmad97/my-first-repo.mlflow


'https://dagshub.com/Ahmad97/my-first-repo.mlflow'

In [16]:

print(f"Tracking URI: {mlflow.get_tracking_uri()}")


Tracking URI: https://dagshub.com/Ahmad97/my-first-repo.mlflow
